# Clustering de séries temporelles

## Préparation des données

In [ ]:
df = pd.read_parquet("C:/Users/gabri/OneDrive/Bureau/Cours ENSAE/2ème année/Stage/Données/forecasting.parquet.gzip")
df

On mets les dates au format temps

In [ ]:
df['time_index']=pd.to_datetime(df['time_index'])

On regroupe ensuite les séries temporelles selon l'indice du produit. On effectue une copie puisqu'on va d'abord regarder les résultats d'un clustering avec une distance euclidienne donc on complétera les séries par des zéros et on utilisera ensuite la DTW qui ne nécéssite pas de séries de même taille

In [ ]:
indice_produit=df['product_index'].unique()
myseries=[df[['time_index','ordered_volumes']][df['product_index']==k] for k in indice_produit]
for k in range(len(myseries)):
    myseries[k].set_index('time_index',inplace=True)
    myseries[k].sort_index(inplace=True)
myseries2=myseries.copy()
indice_temps=[series.index for series in myseries2 ]

On complète donc les séries pour qu'elle ait la même taille

In [ ]:
series_length=[len(series) for series in myseries]
max_len = max(series_length)
longest_series = None
for series in myseries:
    if len(series) == max_len:
        longest_series = series
for i in range(len(myseries)):
    if len(myseries[i])!= max_len:
        myseries[i] = myseries[i].reindex(longest_series.index,fill_value=0)

On normalise les séries

In [ ]:
for i in range(len(myseries)):
    scaler = MinMaxScaler()
    myseries[i] = MinMaxScaler().fit_transform(myseries[i])
    myseries[i]= myseries[i].reshape(len(myseries[i]))
    myseries2[i] = MinMaxScaler().fit_transform(myseries2[i])
    myseries2[i]= myseries2[i].reshape(len(myseries2[i]))

## Clustering Kmeans

### Kmeans avec les séries originales

On traçe l'inertie pour savoir le nombre de clusters à choisir

In [ ]:
inertie=np.empty(25,dtype='float')

for i in range(1,26):
    kmeans = KMeans(n_clusters=i,max_iter=5000)
    inertie[i-1] = kmeans.fit(myseries).inertia_
plt.plot(range(1,26),inertie)

Avec la règle du coude, on voit qu'il faudrait choisir entre 3 et 6 clusters.
Pour ici on va prendre 4 clusters.

In [ ]:
kmeans = KMeans(n_clusters=4,max_iter=5000)

labels_kmeans = kmeans.fit_predict(myseries)

On représente les séries d'un cluster en gris sur un même graphe avec tracé en rouge le centroïde

In [ ]:
fig, axs = plt.subplots(1,4,figsize=(25,10))
fig.suptitle('Clusters')
column_j=0
for label in set(labels_kmeans):
    cluster = []
    for i in range(len(labels_kmeans)):
            if(labels_kmeans[i]==label):
                axs[column_j].plot(longest_series.index,myseries[i],c="gray",alpha=0.4)
                cluster.append(myseries[i])
    if len(cluster) > 0:
        axs[ column_j].plot(longest_series.index,np.average(np.vstack(cluster),axis=0),c="red")
    axs[ column_j].set_title("Cluster "+str(column_j))
    column_j+=1     
plt.show()

On peut regarder les indices de chacun des clusters 

In [ ]:
fancy_names_for_labels_kmeans = [f"Cluster {label}" for label in labels_kmeans]
prédiction_kmeans=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_kmeans),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")
for k in range(4):
    print(prédiction_kmeans[prédiction_kmeans['Cluster']==f'Cluster {k}'].index)

### Kmeans avec une ACP

On va effectuer une analyse en composante principale pour voir si les résultats sont similaires
On regarde l'histogramme de la variance expliquée par la n-ième composante pour choisir le nombre de composantes

In [ ]:
pca = PCA(n_components=20)
pca.fit(myseries)
plt.hist(range(1,21),weights=pca.explained_variance_ratio_,bins=20)

On choisit 3 composantes (règle du coude)

In [ ]:
pca = PCA(n_components=3)

myseries_transformed = pca.fit_transform(myseries)

On trace de nouveau l'inertie

In [ ]:
inertie=np.empty(25,dtype='float')

for i in range(1,26):
    kmeans = KMeans(n_clusters=i,max_iter=5000)
    inertie[i-1] = kmeans.fit(myseries_transformed).inertia_
plt.plot(range(1,26),inertie)

On a le même type de courbe, on va donc choisir à nouveau 4 clusters et observer les résultats.

In [ ]:
kmeans_pca = KMeans(n_clusters=4,max_iter=5000)

labels_pca = kmeans_pca.fit_predict(myseries_transformed)

In [ ]:
fig, axs = plt.subplots(1,4,figsize=(25,10))
fig.suptitle('Clusters')
column_j=0
for label in set(labels_pca):
    cluster = []
    for i in range(len(labels_pca)):
            if(labels_pca[i]==label):
                axs[column_j].plot(longest_series.index,myseries[i],c="gray",alpha=0.4)
                cluster.append(myseries[i])
    if len(cluster) > 0:
        axs[ column_j].plot(longest_series.index,np.average(np.vstack(cluster),axis=0),c="red")
    axs[column_j].set_title("Cluster "+str(column_j))
    column_j+=1
 
        
plt.show()

In [ ]:
fancy_names_for_labels_pca = [f"Cluster {label}" for label in labels_pca]
prédiction_pca=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_pca),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")
for k in range(4):
    print(prédiction_pca[prédiction_pca['Cluster']==f'Cluster {k}'].index)

## Clustering DTW

On passe la série au format Time Series pour tslearn

In [ ]:
X=to_time_series_dataset(myseries2)

### Kmeans 

On peut traçer l'inertie mais ceci prend un peu plus de temps (je laisse quand même le code)

In [ ]:
#inertie=np.empty((1,10),dtype='float')

#for j in range(1):
#    for i in range(1,11):
#        km = TimeSeriesKMeans(n_clusters=i, metric="dtw")
#        inertie[j,i-1] = km.fit(X).inertia_
#    print(j)
#plt.plot(range(1,11),np.mean(inertie,axis=0))

La courbe est assez similaire mais on va s'intéresser par la suite à 3 clusters

In [ ]:
dtw_km = TimeSeriesKMeans(n_clusters=3, metric="dtw")

labels_dtw = dtw_km.fit_predict(X)

In [ ]:
fancy_names_for_labels_dtw = [f"Cluster {label}" for label in labels_dtw]
prediction_dtw=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_dtw),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")

On regarde les séries de chaque cluster (il faut redimensionner la deuxième coordonnée de figsize en fonction des résultats pour avoir des cases qui ne sont pas trop écrasées ou allongées)

In [ ]:
fig, axs = plt.subplots(len(prediction_dtw[prediction_dtw['Cluster']==f'Cluster {0}'].index)//4+1,4,figsize=(25,15))
row_i=0
column_j=0
for i in prediction_dtw[prediction_dtw['Cluster']==f'Cluster {0}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show

In [ ]:
fig, axs = plt.subplots(len(prediction_dtw[prediction_dtw['Cluster']==f'Cluster {1}'].index)//4+1,4,figsize=(25,25))
row_i=0
column_j=0
for i in prediction_dtw[prediction_dtw['Cluster']==f'Cluster {1}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show

In [ ]:
fig, axs = plt.subplots(len(prediction_dtw[prediction_dtw['Cluster']==f'Cluster {2}'].index)//4+1,4,figsize=(25,10))
row_i=0
column_j=0
for i in prediction_dtw[prediction_dtw['Cluster']==f'Cluster {2}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show

In [ ]:
for k in range(3):
    print(prediction_dtw[prediction_dtw['Cluster']==f'Cluster {k}'].index)

### Hierarchical clustering

On utilise maintenant l'algorithme de hierarchical clustering et on trace les dendrogrammes pour chaque type de lien. Comme il n'existe pas de fonction pour faire cet algorithme avec la DTW, on va calculer les distances avant

In [ ]:
dtw_precomputed=np.zeros((125,125))
for i in range(125):
    for j in range(125):
        dtw_precomputed[i,j]=dtw(myseries2[i], myseries2[j])

In [ ]:
hac_average=AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='average',compute_distances=True)
hac_single=AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='single',compute_distances=True)
hac_complete=AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete',compute_distances=True)
modele_average = hac_average.fit(dtw_precomputed)
modele_single = hac_single.fit(dtw_precomputed)
modele_complete = hac_complete.fit(dtw_precomputed)

In [ ]:
def plot_dendrogram(model, **kwargs):
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)


    dendrogram(linkage_matrix, **kwargs)

In [ ]:
plot_dendrogram(modele_average, truncate_mode="level", p=5)

In [ ]:
plot_dendrogram(modele_single, truncate_mode="level", p=10)

In [ ]:
plot_dendrogram(modele_complete, truncate_mode="level", p=4)

Au vu des dendrogrammes, on va utilise le linkage complete. On va faire de nouveau 3 clusters 

In [ ]:
hac=AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete')

labels_hac = hac.fit_predict(dtw_precomputed)

In [ ]:
fancy_names_for_labels_hac = [f"Cluster {label}" for label in labels_hac]
prediction_hac=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_hac),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")

On observe les séries pour chaque cluster

In [ ]:
ig, axs = plt.subplots(len(prediction_hac[prediction_hac['Cluster']==f'Cluster {0}'].index)//4+1,4,figsize=(25,35))
row_i=0
column_j=0
for i in prediction_hac[prediction_hac['Cluster']==f'Cluster {0}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show

In [ ]:
ig, axs = plt.subplots(len(prediction_hac[prediction_hac['Cluster']==f'Cluster {0}'].index)//4+1,4,figsize=(25,35))
row_i=0
column_j=0
for i in prediction_hac[prediction_hac['Cluster']==f'Cluster {1}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show

In [ ]:
ig, axs = plt.subplots(len(prediction_hac[prediction_hac['Cluster']==f'Cluster {0}'].index)//4+1,4,figsize=(25,35))
row_i=0
column_j=0
for i in prediction_hac[prediction_hac['Cluster']==f'Cluster {2}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show

In [ ]:
for k in range(3):
    print(prediction_hac[prediction_hac['Cluster']==f'Cluster {k}'].index)

### DBSCAN

On estime d'abord les paramètres de l'algorithme. La distance e est choisie pour qu'une grande partie des points ait la distance minimale inférieure à ce point. Le nombre n est choisi de telle sorte que le nombre de voisin à une distance inférieure à n pour un nombre suffisamment grand de point. Ici, on essaye les quantiles à 95 % et à 90%

In [ ]:
a=np.copy(dtw_precomputed)
e=np.quantile(np.sort(a,axis=0)[1],0.95)
n=np.quantile(np.count_nonzero(a<=e,axis=0),0.05)
e2=np.quantile(np.sort(a,axis=0)[1],0.9)
n2=np.quantile(np.count_nonzero(a<=e2,axis=0),0.1)

On regarde ici les résultats pour 95%

In [ ]:
dbscan=DBSCAN(eps=e,min_samples=n,metric='precomputed')
labels_dbscan = dbscan.fit_predict(dtw_precomputed)
fancy_names_for_labels_dbscan= [f"Cluster {label}" for label in labels_dbscan]
prediction_dbscan=pd.DataFrame(zip(indice_produit,fancy_names_for_labels_dbscan),columns=["Series","Cluster"]).sort_values(by="Cluster").set_index("Series")

In [ ]:
prediction_dbscan

In [ ]:
for k in range(-1,2):
    print(prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {k}'].index)

In [ ]:
fig, axs = plt.subplots(len(prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {-1}'].index)//4+1,4,figsize=(25,50))
row_i=0
column_j=0
for i in prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {-1}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show

In [ ]:
fig, axs = plt.subplots(len(prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {0}'].index)//4+1,4,figsize=(25,50))
row_i=0
column_j=0
for i in prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {0}'].index:
    axs[row_i,column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show

In [ ]:
fig, axs = plt.subplots(len(prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {1}'].index)//4+1,4,figsize=(25,5))
row_i=0
column_j=0
for i in prediction_dbscan[prediction_dbscan['Cluster']==f'Cluster {1}'].index:
    axs[column_j].plot(indice_temps[np.where(indice_produit==i)[0][0]],myseries2[np.where(indice_produit==i)[0][0]],c="blue")
    column_j+=1
    if column_j%4==0:
        row_i+=1
        column_j=0
        
plt.show

## Comparaison des modèles

### Comparaison inertie entre les algorithmes Kmeans

On regarde l'inertie pour les différents modèles utilisant l'algorithme des kmeans. On rappelle qu'on préfère une petite inertie pour un petit nombre de clusters

In [ ]:
modeles_inertie=np.array([[KMeans(n_clusters=3,max_iter=5000).fit(myseries_transformed).inertia_,KMeans(n_clusters=4,max_iter=5000).fit(myseries_transformed3).inertia_,KMeans(n_clusters=5,max_iter=5000).fit(myseries_transformed).inertia_,KMeans(n_clusters=6,max_iter=5000).fit(myseries_transformed).inertia_],
                     [KMeans(n_clusters=3,max_iter=5000).fit(myseries).inertia_,KMeans(n_clusters=4,max_iter=5000).fit(myseries).inertia_,KMeans(n_clusters=5,max_iter=5000).fit(myseries).inertia_,KMeans(n_clusters=6,max_iter=5000).fit(myseries).inertia_],
                      [TimeSeriesKMeans(n_clusters=3, metric="dtw").fit(X).inertia_,TimeSeriesKMeans(n_clusters=4, metric="dtw").fit(X).inertia_,TimeSeriesKMeans(n_clusters=5, metric="dtw").fit(X).inertia_,TimeSeriesKMeans(n_clusters=6, metric="dtw").fit(X).inertia_]])



In [ ]:
fig, ax =plt.subplots(1,1,figsize=(10,10))

column_labels=["n=3", "n=4", "n=5","n=6"]
row_labels=["PCA 3 Composantes","Kmeans","DTW"]
ax.axis('tight')
ax.axis('off')
ax.table(cellText=modeles_inertie,colLabels=column_labels,rowLabels=row_labels,loc="center")

plt.show()

On voit que l'algorithme avec la DTW est largement meilleure

### Comparaison algorithmes utilisant la DTW

#### Silhouette score

On regarde le silhouette score qui est optimal quand il est proche de 1. On ne regarde que les algorithmes utilisant la DTW, on regarde le Kmeans, hierarchical clustering avec average et complete linkage et DBSCAN avec les deux paires de paramètres calculées avant

In [ ]:
modeles_silhouette=np.zeros((5,4))
a=silhouette_score2(X,DBSCAN(eps=e,min_samples=n,metric='precomputed').fit_predict(dtw_precomputed))
b=silhouette_score2(X,DBSCAN(eps=e2,min_samples=n2,metric='precomputed').fit_predict(dtw_precomputed))
c=np.array([silhouette_score2(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed))])
d= np.array([silhouette_score2(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),silhouette_score2(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed))])
modeles_silhouette[0]=np.array([silhouette_score2(X,TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X)),silhouette_score2(X,TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X)),silhouette_score2(X,TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X)),silhouette_score2(X,TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X))])
modeles_silhouette[1]=c
modeles_silhouette[2]=d                                
modeles_silhouette[3]=np.full(4,a)
modeles_silhouette[4]=np.full(4,b)


In [ ]:
fig, ax =plt.subplots(1,1,figsize=(10,10))

column_labels=["n=3", "n=4", "n=5","n=6"]
row_labels=["DTW","HAC Complete","HAC average","DBSCAN eps=1,82 min_samples=2","DBSCAN eps=1,72 min_samples=2"]
ax.axis('tight')
ax.axis('off')
ax.table(cellText=modeles_silhouette,colLabels=column_labels,rowLabels=row_labels,loc="center")

plt.show()

#### Score de Calinski-Harabasz

On regarde le score de Calinski-Harabasz qui va de 0 à l'infini où l'infini est la meilleure classification

In [ ]:
def BGSS(X,labels):
    bgss=0
    C=dtw_barycenter_averaging(X)
    for k in np.unique(labels):
        n_k=len(np.where(labels==k)[0])
        C_k=dtw_barycenter_averaging(X[np.where(labels==k)])
        bgss+=n_k*dtw(C_k,C)**2
    return bgss

In [ ]:
def WGSS(X,labels):
    WGSS=0
    for k in np.unique(labels):
        WGSSk=0
        C_k=dtw_barycenter_averaging(X[np.where(labels==k)])
        for x in np.where(labels==k)[0]:
            WGSSk+=dtw(C_k,X[x])**2
        WGSS+=WGSSk
    return WGSS

In [ ]:
def Calinski_Harabasz_score(X,labels):
    N=len(X)
    K=len(np.unique(labels))
    return BGSS(X,labels)*(N-K)/(WGSS(X,labels)*(K-1))

In [ ]:
modeles_calinski_harabasz=np.zeros((1,5,4))
a=Calinski_Harabasz_score(X,DBSCAN(eps=e,min_samples=n,metric='precomputed').fit_predict(dtw_precomputed))
b=Calinski_Harabasz_score(X,DBSCAN(eps=e2,min_samples=n2,metric='precomputed').fit_predict(dtw_precomputed))
c=np.array([Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed))])
d=np.array([Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Calinski_Harabasz_score(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed))])
for k in range(1):
    modeles_calinski_harabasz[k,0]=np.array([Calinski_Harabasz_score(X,TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X)),Calinski_Harabasz_score(X,TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X)),Calinski_Harabasz_score(X,TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X)),Calinski_Harabasz_score(X,TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X))])
    modeles_calinski_harabasz[k,1]=c
    modeles_calinski_harabasz[k,2]=d                                  
    modeles_calinski_harabasz[k,3]=np.full(4,a)
    modeles_calinski_harabasz[k,4]=np.full(4,b)


In [ ]:
fig, ax =plt.subplots(1,1,figsize=(10,10))
column_labels=["n=3", "n=4", "n=5","n=6"]
row_labels=["DTW","HAC Complete","HAC average","DBSCAN eps=1,82 min_samples=2","DBSCAN eps=1,72 min_samples=2"]
ax.axis('tight')
ax.axis('off')
ax.table(cellText=np.mean(modeles_calinski_harabasz, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

plt.show()

#### Score de Davies-Bouldin

On regarde le score de Davies-Bouldin qui va de 0 à l'infini où 0 est la meilleure classification

In [ ]:
def distance_intra_cluster(X,labels,k):
    d=0
    C_k=dtw_barycenter_averaging(X[np.where(labels==k)])
    for x in np.where(labels==k)[0]:
            d+=dtw(C_k,X[x])
    return d

In [ ]:
def Davies_Bouldin_score(X,labels):
    K=len(np.unique(labels))
    score=0
    for k in np.unique(labels):
        maxi=0
        for k2 in np.unique(labels):
            if k2!=k:
                if (distance_intra_cluster(X,labels,k)+distance_intra_cluster(X,labels,k2))/dtw(dtw_barycenter_averaging(X[np.where(labels==k)]),dtw_barycenter_averaging(X[np.where(labels==k2)]))>maxi:
                    maxi=(distance_intra_cluster(X,labels,k)+distance_intra_cluster(X,labels,k2))/dtw(dtw_barycenter_averaging(X[np.where(labels==k)]),dtw_barycenter_averaging(X[np.where(labels==k2)]))
        score+=maxi
    return score

In [ ]:
modeles_davies_bouldin=np.zeros((1,5,4))
a=Davies_Bouldin_score(X,DBSCAN(eps=e,min_samples=n,metric='precomputed').fit_predict(dtw_precomputed))
b=Davies_Bouldin_score(X,DBSCAN(eps=e2,min_samples=n2,metric='precomputed').fit_predict(dtw_precomputed))
c=np.array([Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed))])
d=np.array([Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=4,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=5,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed)),Davies_Bouldin_score(X,AgglomerativeClustering(n_clusters=6,affinity='precomputed',linkage='average').fit_predict(dtw_precomputed))])
for k in range(1):
    modeles_davies_bouldin[k,0]=np.array([Davies_Bouldin_score(X,TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X)),Davies_Bouldin_score(X,TimeSeriesKMeans(n_clusters=4, metric="dtw").fit_predict(X)),Davies_Bouldin_score(X,TimeSeriesKMeans(n_clusters=5, metric="dtw").fit_predict(X)),Davies_Bouldin_score(X,TimeSeriesKMeans(n_clusters=6, metric="dtw").fit_predict(X))])
    modeles_davies_bouldin[k,1]=c
    modeles_davies_bouldin[k,2]=d                                  
    modeles_davies_bouldin[k,3]=np.full(4,a)
    modeles_davies_bouldin[k,4]=np.full(4,b)



In [ ]:
fig, ax =plt.subplots(1,1,figsize=(10,10))

column_labels=["n=3", "n=4", "n=5","n=6"]
row_labels=["DTW","HAC Complete","HAC average","DBSCAN eps=1,82 min_samples=2","DBSCAN eps=1,72 min_samples=2"]
ax.axis('tight')
ax.axis('off')
ax.table(cellText=np.mean(modeles_davies_bouldin, axis=0),colLabels=column_labels,rowLabels=row_labels,loc="center")

plt.show()

### Indices se retrouvant tout le temps ensemble ou jamais ensemble

On va regarder les indices que l'on retrouvera souvent ensemble. On va appliquer plusieurs fois le kmeans avec la DTW et le hierarchical clustering avec un linkage complete puisque ce sont ceux qui ont le meilleur résultat sur les scores précédents

In [ ]:
def toujours_ensemble(i,j,*args):
    for k in args:
        if k[i]!=k[j]:
            return False
    return True

In [ ]:
def dans_liste_de_liste(l,i):
    for k in l:
        if i in k:
            return True
    return False

In [ ]:
def indices_toujours_ensemble(*args):
    l=[]
    for i in range(125):
        li=[]
        if not dans_liste_de_liste(l,indice_produit[i]):
            for j in range(125):
                if toujours_ensemble(i,j,*args):
                    li.append(indice_produit[j])
            if len(li)>1:
                l.append(li)
    return l

In [ ]:
c_trois=TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X)

In [ ]:
argument3=c_trois,TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),TimeSeriesKMeans(n_clusters=3, metric="dtw").fit_predict(X),AgglomerativeClustering(n_clusters=3,affinity='precomputed',linkage='complete').fit_predict(dtw_precomputed)

In [ ]:
a=indices_toujours_ensemble(*argument3)

On peut aussi regarder les indices que l'on ne retrouve jamais ensemble mais cela donne une matrice qui est un peu moins lisible puisque pour chaque indice on a la liste des éléments qui ne sont pas avec lui

In [ ]:
def jamais_ensemble(i,j,*args):
    for k in args:
        if k[i]==k[j]:
            return False
    return True

In [ ]:
def indices_jamais_avec_k(k,*args):
    l=[k]
    i=np.where(indice_produit==k)[0][0]
    for j in range(125):
        if jamais_ensemble(i,j,*args):
            l.append(indice_produit[j])
    return l

In [ ]:
%%time
liste3=[]
for k in indice_produit:
    liste3.append(indices_jamais_avec_k(k,*argument3))
liste3